In [13]:
%load_ext autoreload
%autoreload 2

In [12]:
from datasets import load_dataset

# 2. Load the dataset

ds = load_dataset("dragonslayer631/ci2_allsides", split="train")

In [2]:
# # 3. Get the text data (TEST)

# import pandas as pd

# df = pd.read_csv("data/summary_text_sentiment.csv", sep="|")
# df.set_index("id", inplace=True)

In [14]:
from utils.groq_utils import generate_topic_similarity_groq, get_groq_client

client = get_groq_client()

In [ ]:
import json

def sentiment_to_buckets(row, text, model=None):
    topics = row['tags'].split(", ")
    entity_sentiments = json.loads(row[f'{text}_entity_sentiments'])
    entities = [k for k, v in entity_sentiments.items() if v != 0]
    
    resp = generate_topic_similarity_groq(topics=topics, entities=entities, client=client, model=model)
    
    topics_to_sentiments = {}
    
    for topic, entities in resp.items():
        try:
            if topic not in topics_to_sentiments:
                topics_to_sentiments[topic] = 0
            for entity, value in entities.items():
                if value > 0.7 and entity in entity_sentiments.keys():
                    topics_to_sentiments[topic] += entity_sentiments[entity]
        except Exception as e:
            print(f"groq response: \n{topics_to_sentiments}\n")
            print(f"e={e}")
            return json.dumps({})
    
    return json.dumps(topics_to_sentiments)

In [17]:
new_values = {}

In [30]:
def get_topic_sentiments_for_text_in_dataset(ds: Dataset, text: str):
    ctr = 0
    filtered = ds.filter(lambda x: x[f'{text}_topic_to_sentiment'] == None)
    print(f"processing {len(filtered)} rows")
    for row in filtered:
        if row['id'] in new_values.keys():
            continue

        ctr += 1
        if ctr % 100 == 0:
            print(f"Processed {ctr} rows")
        try:
            topics_to_sentiments = sentiment_to_buckets(row=row, text=text, model="meta-llama/llama-4-scout-17b-16e-instruct")
            with open(f"data/{text}/{text}_topic_to_sim.csv", "a") as f:
                f.write(f"{row['id']}|{topics_to_sentiments}\n")
            new_values[row['id']] = topics_to_sentiments
        except Exception as e:
            print(f"Error processing row {row['id']}: {e}")


In [80]:
type = "summary_50"
get_topic_sentiments_for_text_in_dataset(ds, type)
update_column = f'{type}_topic_to_sentiment'

processing 38960 rows


In [81]:
len(new_values)

38863

In [82]:
from utils.ds_utils import load_dataset_from_huggingface

ds = load_dataset_from_huggingface()

Generating train split: 100%|██████████| 45089/45089 [00:00<00:00, 45547.99 examples/s]

Loaded dataset: dragonslayer631/ci2_allsides, split: train


In [84]:
ds[0]

{'topic': 'economy_and_jobs',
 'tags': 'Economy And Jobs, Economic Policy, Budget Reconciliation, Infrastructure, Infrastructure Bill, Debt Ceiling',
 'text': 'We use cookies and similar methods to recognize visitors and remember their preferences. We may also use them to measure ad campaign effectiveness, target ads, and analyze site traffic. Depending on your location, you may opt-in or opt out of the use of these technologies. Search Illustration: Sarah Grillo/Axios The Senate\'s bipartisan $1.2 trillion infrastructure bill and President Biden\'s $3.5 trillion spending package could live or die this week — and take Democrats\' fortunes with them. But all the minute-by-minute political drama obscures how much America could change if even a fraction of it passes. The big picture: Anything short of total failure could have a transformative impact on day-to-day life — from how we move around to our access to the internet, paid family leave and child care, health care and college. Here\'

In [85]:
def update_dataset_column(row):
    if row["id"] in new_values.keys():
        row[update_column] = new_values[row["id"]]
    return row

In [86]:
updated = ds.map(update_dataset_column)

Map: 100%|██████████| 45089/45089 [00:13<00:00, 3348.27 examples/s]


In [87]:
filtered = updated.filter(lambda x: x[update_column] == None)

filtered

Filter: 100%|██████████| 45089/45089 [00:38<00:00, 1160.46 examples/s]


Dataset({
    features: ['topic', 'tags', 'text', 'int_bias', 'summary_5', 'summary_50', 'summary_100', 'text_entity_sentiments', 'text_topic_to_sentiment', 'summary_100_entity_sentiments', 'summary_100_topic_to_sentiment', 'summary_50_entity_sentiments', 'summary_50_topic_to_sentiment', 'id', 'text_encoded', 'summary_100_encoded', 'summary_50_encoded', 'text_topic_0', 'text_sentiment_0', 'text_topic_1', 'text_sentiment_1', 'text_topic_2', 'text_sentiment_2', 'text_topic_3', 'text_sentiment_3', 'text_topic_4', 'text_sentiment_4', 'summary_100_topic_0', 'summary_100_sentiment_0', 'summary_100_topic_1', 'summary_100_sentiment_1', 'summary_100_topic_2', 'summary_100_sentiment_2', 'summary_100_topic_3', 'summary_100_sentiment_3', 'summary_100_topic_4', 'summary_100_sentiment_4'],
    num_rows: 0
})

In [89]:
ds[0]

{'topic': 'economy_and_jobs',
 'tags': 'Economy And Jobs, Economic Policy, Budget Reconciliation, Infrastructure, Infrastructure Bill, Debt Ceiling',
 'text': 'We use cookies and similar methods to recognize visitors and remember their preferences. We may also use them to measure ad campaign effectiveness, target ads, and analyze site traffic. Depending on your location, you may opt-in or opt out of the use of these technologies. Search Illustration: Sarah Grillo/Axios The Senate\'s bipartisan $1.2 trillion infrastructure bill and President Biden\'s $3.5 trillion spending package could live or die this week — and take Democrats\' fortunes with them. But all the minute-by-minute political drama obscures how much America could change if even a fraction of it passes. The big picture: Anything short of total failure could have a transformative impact on day-to-day life — from how we move around to our access to the internet, paid family leave and child care, health care and college. Here\'

In [90]:
ds = updated
new_values = {}

In [91]:
from utils.ds_utils import save_dataset_to_huggingface

save_dataset_to_huggingface(ds)

Uploading the dataset shards: 100%|██████████| 2/2 [09:09<00:00, 274.95s/it]


Saved dataset: dragonslayer631/ci2_allsides, split: train
